# Tugas 1 : Crawling Berita

**Pengertian Crawling**


Crawling adalah proses otomatisasi yang dilakukan oleh program komputer untuk menjelajahi dan mengumpulkan data dari halaman-halaman web di internet. Proses ini sering kali dilakukan oleh bot yang dikenal sebagai web crawlers atau spiders. Web crawlers ini akan menelusuri (crawl) berbagai situs web, mengakses halaman-halaman yang ada, dan mengunduh atau mengekstraksi informasi yang dibutuhkan untuk kemudian disimpan atau diindeks dalam database.

**Fungsi Crawling**



1.   Mengindeks Halaman Web:


*   
Crawling adalah langkah pertama dalam proses pengindeksan halaman web oleh mesin pencari. Web crawler akan mengunjungi halaman-halaman web, membaca kontennya, dan menyimpannya ke dalam indeks mesin pencari. Ini memungkinkan mesin pencari untuk menemukan dan menampilkan halaman-halaman tersebut dalam hasil pencarian.



2.   Pengumpulan Data:

*   Crawling memungkinkan pengumpulan data dari berbagai situs web untuk keperluan tertentu, seperti analisis bisnis, riset pasar, atau pengembangan model kecerdasan buatan. Dengan crawling, data dari berbagai sumber bisa dikumpulkan secara otomatis tanpa harus melakukannya secara manual.



3. Pemantauan Perubahan Konten:

*   Dengan crawling, perubahan atau pembaruan pada suatu situs web dapat dipantau secara berkala. Ini berguna untuk aplikasi yang memerlukan informasi terbaru, seperti agregator berita, alat pemantau harga, atau layanan notifikasi.



4. SEO (Search Engine Optimization):

*   Bagi pengelola situs web, memahami bagaimana proses crawling bekerja dapat membantu dalam optimasi mesin pencari (SEO). Dengan memastikan bahwa halaman-halaman web mereka mudah di-crawl dan diindeks, mereka bisa meningkatkan kemungkinan situs web mereka muncul di hasil pencarian mesin pencari.


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://health.detik.com/berita-detikhealth/indeks"]
categories = ["Kesehatan"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)


Merinding! Penampakan Cacing Pita di Kaki Pasien yang Makan Daging Babi Tak Matang
Kondisi Terkini Istri Ridwan Kamil Pasca Positif COVID-19
6 Rekomendasi Makanan Tinggi Serat untuk Menurunkan BB, Enak dan Mudah Didapat
Epidemiolog Nilai Pelabelan Bahaya BPA di Galon AMDK Tepat & Penting
Ini yang Terjadi pada Tubuh Jika Kolesterol Tinggi Tak Segera Ditangani
Pria di India Telan Pisau Kecil hingga Gunting Kuku, Diduga Idap Kondisi Ini
Tak Ada Vaksinasi Massal untuk Mpox, Pakar Jelaskan Bedanya dengan COVID-19
Klarifikasi RS Medistra usai Heboh Larangan Dokter dan Perawat Pakai Hijab
7 Makanan Penurun Kolesterol, Alpukat hingga Kacang-kacangan
Kenapa Penamaan Virus Mpox Pakai Istilah 'Clade'? Ini Penjelasan Ilmiahnya


In [16]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,tanggal,isi
0,Merinding! Penampakan Cacing Pita di Kaki Pasi...,"Senin, 02 Sep 2024 14:02 WIB",Jakarta - Seorang dokter di unit gawat darurat...
1,Kondisi Terkini Istri Ridwan Kamil Pasca Posit...,"Senin, 02 Sep 2024 13:30 WIB",Jakarta - Istri bakal calon gubernur Jakarta R...
2,6 Rekomendasi Makanan Tinggi Serat untuk Menur...,"Senin, 02 Sep 2024 13:02 WIB",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tJenis Maka...
3,Epidemiolog Nilai Pelabelan Bahaya BPA di Galo...,"Senin, 02 Sep 2024 12:23 WIB",Jakarta - Epidemiolog Dicky Budiman menyoroti ...
4,Ini yang Terjadi pada Tubuh Jika Kolesterol Ti...,"Senin, 02 Sep 2024 12:02 WIB",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\t1. Penyaki...
5,Pria di India Telan Pisau Kecil hingga Gunting...,"Senin, 02 Sep 2024 11:34 WIB","Jakarta - Di sebuah rumah sakit di Motihari, B..."
6,"Tak Ada Vaksinasi Massal untuk Mpox, Pakar Jel...","Senin, 02 Sep 2024 10:57 WIB",Jakarta -\n \n Kementerian Keseh...
7,Klarifikasi RS Medistra usai Heboh Larangan Do...,"Senin, 02 Sep 2024 10:03 WIB",Jakarta - Viral di media sosial RS Medistra di...
8,"7 Makanan Penurun Kolesterol, Alpukat hingga K...","Senin, 02 Sep 2024 09:30 WIB",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tMakanan Pe...
9,Kenapa Penamaan Virus Mpox Pakai Istilah 'Clad...,"Senin, 02 Sep 2024 09:01 WIB",Jakarta - Sedikitnya ada 2 jenis Monkeypox Vir...
